# Prediction of MRS effects

In [15]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNetCV
import os
import copy

In [2]:
# read in MRS data
glu_res_data = pd.read_csv('C:/Users/canlab/Desktop/Masterthesis_python/Jenny_glutamate_and_gm_values.csv')

In [3]:
# glu_tcr: total creatine - creatine + phosphocreatine
# gm: grey matter ratio in the MRS voxel of the subject
# glu_tcr_baseline_corrected/24h_corrected: residuals + intercept for the model glu_cr.... ~ gm_ratio
glu_res_data

,ID,treatment,glu_tcr_baseline,glu_tcr_24h,gm_baseline,gm_24h,glu_tcr_baseline_corrected,glu_tcr_24h_corrected,difference_24h_baseline
0,ap75,Placebo,0.684,0.640,0.7028,0.6806,0.650491,0.634923,-0.015568
1,bg52,Verum,0.769,0.387,0.7178,0.7392,0.734776,0.381486,-0.353290
2,bh98,Verum,0.675,0.678,0.7579,0.7739,0.638864,0.672227,0.033363
3,bu45,Placebo,0.751,0.609,0.8396,0.8545,0.710969,0.602626,-0.108343
4,ce22,Verum,0.633,0.645,0.7326,0.8173,0.598071,0.638904,0.040833
5,cu52,Verum,0.563,0.689,0.7600,0.7108,0.526764,0.683698,0.156934
6,dp20,Placebo,0.591,0.552,0.8124,0.7780,0.552266,0.546197,-0.006069
7,dp37,Verum,0.398,0.660,0.7808,0.7754,0.360773,0.654216,0.293444
8,dp92,Placebo,0.667,0.631,0.7267,0.7110,0.632352,0.625697,-0.006655
9,dt27,Placebo,0.643,0.647,0.8340,0.7742,0.603236,0.641225,0.037989


In [18]:
ket = copy.deepcopy(glu_res_data[glu_res_data['treatment'] == 'Verum'])

In [19]:
placebo = copy.deepcopy(glu_res_data[glu_res_data['treatment'] == 'Placebo'])

In [9]:
ket.count()

ID                            27
treatment                     27
glu_tcr_baseline              27
glu_tcr_24h                   27
gm_baseline                   27
gm_24h                        27
glu_tcr_baseline_corrected    27
glu_tcr_24h_corrected         27
difference_24h_baseline       27
dtype: int64

In [10]:
placebo.count()

ID                            26
treatment                     26
glu_tcr_baseline              26
glu_tcr_24h                   26
gm_baseline                   26
gm_24h                        26
glu_tcr_baseline_corrected    26
glu_tcr_24h_corrected         26
difference_24h_baseline       26
dtype: int64

In [12]:
fc_feat = np.load('C:/Users/canlab/Desktop/Masterthesis_python/ket/FC_features-bg52', allow_pickle=True)

In [13]:
fc_feat

,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,-0.060794,-0.039162,-0.030193,-0.030875,-0.006244,-0.014368,0.109128,-0.069767,0.002359,-0.037030
33,-0.016118,-0.010245,-0.008499,-0.015682,-0.025337,-0.018579,0.016135,0.019050,-0.019523,0.010395
Caudate,0.018753,-0.004139,-0.005690,0.010057,-0.006822,-0.001468,-0.045082,0.024679,0.032477,-0.012345
FP2,0.041885,-0.040878,0.020763,0.027150,0.022503,-0.061360,-0.057410,0.040132,-0.044452,-0.039697
NAcc,-0.038510,-0.050074,-0.025111,-0.009234,0.024019,0.023853,0.017390,-0.001289,0.024356,-0.024291
OFC1,-0.095958,-0.063177,0.011704,-0.090524,0.007438,0.051886,0.117975,-0.033560,-0.025783,-0.001090
OFC2,-0.009966,-0.020165,0.011647,-0.033929,-0.009588,0.008262,0.064357,-0.018491,-0.016428,-0.017696
OFC3,-0.020115,-0.026960,-0.001697,-0.026784,-0.002077,0.016961,0.052406,-0.012243,-0.004439,-0.009416
OFC4,-0.071011,-0.023258,-0.043145,-0.025431,0.019256,0.048199,0.058971,-0.048058,0.005126,-0.028535


In [16]:
# complete subjects list
subj_list = copy.deepcopy(glu_res_data['ID'])

In [20]:
ket_subj_list=copy.deepcopy(ket['ID'])
placebo_subj_list=copy.deepcopy(placebo['ID'])

In [21]:
ket_subj_list

1     bg52
2     bh98
4     ce22
5     cu52
7     dp37
14    fy78
16    gs63
18    hw95
19    hy97
20    ia24
21    ia45
23    ii39
24    il04
25    ju09
26    jw73
28    kd21
31    kw91
35    op18
38    px00
39    qe30
41    sr21
43    tt42
44    uf97
45    vg84
48    wo30
49    ww25
52    yp84
Name: ID, dtype: object

In [62]:
def create_X(subj_list, treatment):
    
    # initialize the matrix containing all subjects
    X_input = []
    # choose path according to saving directory dependend on treatment
    if treatment=='ket':
        path = 'C:/Users/canlab/Desktop/Masterthesis_python/ket/FC_features-'
    else:
        path = 'C:/Users/canlab/Desktop/Masterthesis_python/placebo/FC_features-'
        
    print(path)
    
    
    for subj in subj_list:
        
        subj_name= str(subj)
        subj_path = path+subj_name
        # load features
        fc_feature = np.load(subj_path, allow_pickle=True)
        matrix = fc_feature.to_numpy()
        
        # flatten the matrix
        flatten_matrix = matrix.flatten()
        
        # fill new matrix with all participants
        X_input.append(flatten_matrix)
        
    return X_input


In [63]:
create_X(ket_subj_list, 'ket')

C:/Users/canlab/Desktop/Masterthesis_python/ket/FC_features-


[array([-0.06079367, -0.03916185, -0.03019292, -0.03087469, -0.00624365,
        -0.0143679 ,  0.10912839, -0.06976707,  0.00235945, -0.03702953,
        -0.01611801, -0.0102447 , -0.00849888, -0.01568205, -0.02533721,
        -0.0185793 ,  0.01613469,  0.01904978, -0.01952311,  0.01039545,
         0.01875256, -0.00413903, -0.00568985,  0.01005657, -0.00682218,
        -0.00146831, -0.04508185,  0.02467877,  0.03247745, -0.01234487,
         0.04188536, -0.04087759,  0.02076347,  0.02715022,  0.02250312,
        -0.06136047, -0.05741029,  0.04013157, -0.04445184, -0.03969656,
        -0.03851013, -0.05007369, -0.02511111, -0.00923399,  0.02401901,
         0.02385275,  0.01738958, -0.00128913,  0.02435559, -0.02429091,
        -0.0959578 , -0.06317676,  0.01170421, -0.09052371,  0.00743764,
         0.05188638,  0.11797459, -0.03355993, -0.02578285, -0.00108953,
        -0.00996566, -0.02016543,  0.01164655, -0.03392912, -0.00958755,
         0.00826171,  0.06435685, -0.0184907 , -0.0

In [64]:
create_X(placebo_subj_list, 'placebo')

C:/Users/canlab/Desktop/Masterthesis_python/placebo/FC_features-


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/canlab/Desktop/Masterthesis_python/placebo/FC_features-jy99'

In [22]:
ex = fc_feat.to_numpy()

In [27]:
ex

array([[-0.06079367, -0.03916185, -0.03019292, -0.03087469, -0.00624365,
        -0.0143679 ,  0.10912839, -0.06976707,  0.00235945, -0.03702953],
       [-0.01611801, -0.0102447 , -0.00849888, -0.01568205, -0.02533721,
        -0.0185793 ,  0.01613469,  0.01904978, -0.01952311,  0.01039545],
       [ 0.01875256, -0.00413903, -0.00568985,  0.01005657, -0.00682218,
        -0.00146831, -0.04508185,  0.02467877,  0.03247745, -0.01234487],
       [ 0.04188536, -0.04087759,  0.02076347,  0.02715022,  0.02250312,
        -0.06136047, -0.05741029,  0.04013157, -0.04445184, -0.03969656],
       [-0.03851013, -0.05007369, -0.02511111, -0.00923399,  0.02401901,
         0.02385275,  0.01738958, -0.00128913,  0.02435559, -0.02429091],
       [-0.0959578 , -0.06317676,  0.01170421, -0.09052371,  0.00743764,
         0.05188638,  0.11797459, -0.03355993, -0.02578285, -0.00108953],
       [-0.00996566, -0.02016543,  0.01164655, -0.03392912, -0.00958755,
         0.00826171,  0.06435685, -0.0184907 

In [25]:
flatten=ex.flatten()

In [26]:
flatten

array([-0.06079367, -0.03916185, -0.03019292, -0.03087469, -0.00624365,
       -0.0143679 ,  0.10912839, -0.06976707,  0.00235945, -0.03702953,
       -0.01611801, -0.0102447 , -0.00849888, -0.01568205, -0.02533721,
       -0.0185793 ,  0.01613469,  0.01904978, -0.01952311,  0.01039545,
        0.01875256, -0.00413903, -0.00568985,  0.01005657, -0.00682218,
       -0.00146831, -0.04508185,  0.02467877,  0.03247745, -0.01234487,
        0.04188536, -0.04087759,  0.02076347,  0.02715022,  0.02250312,
       -0.06136047, -0.05741029,  0.04013157, -0.04445184, -0.03969656,
       -0.03851013, -0.05007369, -0.02511111, -0.00923399,  0.02401901,
        0.02385275,  0.01738958, -0.00128913,  0.02435559, -0.02429091,
       -0.0959578 , -0.06317676,  0.01170421, -0.09052371,  0.00743764,
        0.05188638,  0.11797459, -0.03355993, -0.02578285, -0.00108953,
       -0.00996566, -0.02016543,  0.01164655, -0.03392912, -0.00958755,
        0.00826171,  0.06435685, -0.0184907 , -0.01642816, -0.01

In [ ]:
def calculate_elasticNet():
    
    # load individual FC feature matrix
    

In [31]:
subj_k= np.load('C:/Users/canlab/Desktop/Masterthesis_python/New_ketamine_subject_list', allow_pickle=True)
subj_p= np.load('C:/Users/canlab/Desktop/Masterthesis_python/New_placebo_subject_list', allow_pickle=True)

In [33]:
subj=pd.concat([subj_k, subj_p], axis=1)
subj.columns=['ket', 'Placebo']

In [37]:
subj

,ket,Placebo
0,bg52,ap75
1,bh98,bu45
2,ce22,dp20
3,cu52,dp92
4,dp07,dq56
5,dp37,dt27
6,NaN,du02
7,fy78,dw28
8,gs63,em60
9,hw95,ew27


In [55]:
k = pd.DataFrame(ket_subj_list)
k = k.reset_index()

In [40]:
k

,ID
1,bg52
2,bh98
4,ce22
5,cu52
7,dp37
14,fy78
16,gs63
18,hw95
19,hy97
20,ia24


In [56]:
p=pd.DataFrame(placebo_subj_list)
p = p.reset_index()
Louise_list= pd.concat([k,p], axis=1)
p

,index,ID
0,0,ap75
1,3,bu45
2,6,dp20
3,8,dp92
4,9,dt27
5,10,du02
6,11,dw28
7,12,em60
8,13,ew27
9,15,go11


In [59]:
Louise_list.columns=['index', 'K (Louise)','index', ' P (Louise)']

In [60]:
full_compare_list= pd.concat([subj, Louise_list], axis=1, ignore_index=True)

In [61]:
full_compare_list

,0,1,2,3,4,5
0,bg52,ap75,1.0,bg52,0.0,ap75
1,bh98,bu45,2.0,bh98,3.0,bu45
2,ce22,dp20,4.0,ce22,6.0,dp20
3,cu52,dp92,5.0,cu52,8.0,dp92
4,dp07,dq56,7.0,dp37,9.0,dt27
5,dp37,dt27,14.0,fy78,10.0,du02
6,NaN,du02,16.0,gs63,11.0,dw28
7,fy78,dw28,18.0,hw95,12.0,em60
8,gs63,em60,19.0,hy97,13.0,ew27
9,hw95,ew27,20.0,ia24,15.0,go11
